In [5]:
conda install -c conda-forge glpk

Note: you may need to restart the kernel to use updated packages.




==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.3.1

Please update conda by running

    $ conda update -n base -c defaults conda





Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\emok\Anaconda3

  added / updated specs:
    - glpk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    glpk-5.0                   |       h8ffe710_0         3.3 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be INSTALLED:

  glpk               conda-forge/win-64::glpk-5.0-h8ffe710_0 None




glpk-5.0             | 3.3 MB    |            |   0% 
glpk-5.0             | 3.3 MB    |            |   0% 
glpk-5.0             | 3.3 MB    | 7          |   7% 
glpk-5.0             | 3.3 MB    | #2         |  12% 
glpk-5.0             | 3.3 MB    | #9         |  20% 
glpk-5.0             | 3.3 MB    | ##5        |  25% 
glpk-5.0             | 3.3 MB    | ###2       |  33% 
glp

In [4]:
pip install pyomo

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np

from pyomo.environ import *

Static model, when alpha = 0

In [ ]:
# Create a ConcreteModel object
model = ConcreteModel()

# Define the sets
model.Npv = Set()
model.Non = Set()
model.Nof = Set()

# Define the variables


# Define the parameters
model.k_ipv = Param(model.Npv, within=NonNegativeReals)

# Define the objective function
def objective_rule(model):
        return model.

# Define the constraint
def constraint_rule(model):
        
# Create an instance of the model


# Solve the optimization problem
solver = SolverFactory('glpk')
results = solver.solve(instance)

# Check the solver status and retrieve the optimal solution
if str()



Dynamic model, when alpha>0